# Ejercicio 1: Comparando Particiones Usando Modularidad

**Contexto:** En la detección de comunidades, la modularidad es una métrica crucial que nos ayuda a evaluar la calidad de una partición de un grafo. Una partición divide los nodos del grafo en grupos disjuntos (comunidades). Una modularidad más alta generalmente indica que la partición tiene una mejor estructura comunitaria, con densas conexiones dentro de las comunidades y escasas conexiones entre ellas.

**Tu Tarea:**
Implementa una función en Python llamada `get_better_partition_by_modularity`.
Esta función recibirá:
1.  Un grafo de NetworkX (`graph`).
2.  Dos listas de conjuntos de nodos, representando dos particiones diferentes del grafo (`partition1`, `partition2`). Cada partición es una lista de conjuntos, donde cada conjunto contiene los nodos de una comunidad.

La función deberá:
*   Calcular la modularidad para `partition1` y `partition2` utilizando la función `nx.community.quality.modularity(graph, partition)`.
*   Devolver la partición (la lista de conjuntos de nodos) que tenga el valor de modularidad más alto.
*   Si ambas particiones tienen el mismo valor de modularidad, la función puede devolver `partition1`.

Asegúrate de que tu función maneje correctamente los grafos y las particiones proporcionadas.

In [2]:
import networkx as nx
from typing import List, Set, Any, Optional, Tuple

def get_better_partition_by_modularity(graph: nx.Graph, partition1: List[Set[Any]], partition2: List[Set[Any]]) -> List[Set[Any]]:
    # your code here
    modularidad_1 = nx.community.quality.modularity(graph, partition1)
    modularidad_2 = nx.community.quality.modularity(graph, partition2)
    
    # Devolver la partición con mejor modularidad
    if modularidad_1 >= modularidad_2:
        return partition1
    else:
        return partition2

# Crear un grafo de ejemplo simple
G_test1 = nx.Graph()
G_test1.add_edges_from([(0, 1), (0, 2), (1, 2), (3, 4), (3, 5), (4, 5), (2, 3)])

# Definir dos particiones
partition_A = [{0, 1, 2}, {3, 4, 5}]
partition_B = [{0, 3}, {1, 4}, {2, 5}]

result_partition = get_better_partition_by_modularity(G_test1, partition_A, partition_B)
assert result_partition == partition_A, f"Error: Se esperaba la partición A, pero se obtuvo {result_partition}"

# Ejercicio 2: Louvain vs. Girvan-Newman - ¿Qué Partición Tiene Mejor Modularidad?

**Contexto:**
Los algoritmos de Louvain y Girvan-Newman son métodos para detectar comunidades en redes. La modularidad nos ayuda a cuantificar la calidad de una partición (un conjunto de comunidades).

**Tu Tarea:**
Implementa una función en Python llamada `compare_louvain_girvan_newman`.
Esta función recibirá:
1.  Un grafo de NetworkX (`graph`).
2.  Un parámetro opcional `seed` para el algoritmo de Louvain para asegurar reproducibilidad.

La función deberá:
1.  Obtener la partición del grafo usando el algoritmo de Louvain (`nx.community.louvain_communities`). Convierte los `frozenset`s resultantes a `set`s.
2.  Obtener las particiones generadas por el algoritmo de Girvan-Newman (`nx.community.girvan_newman`). De todas estas particiones, identifica la que tiene la modularidad más alta.
3.  Calcular la modularidad para la partición de Louvain y para la mejor partición de Girvan-Newman.
4.  Devolver la partición (como una lista de `set`s de nodos) que tenga el valor de modularidad más alto.
5.  Si ambas tienen la misma modularidad, devuelve la partición de Louvain.

**Suposiciones:**
*   El grafo de entrada no estará vacío.
*   Los algoritmos generarán particiones válidas.


In [3]:
def compare_louvain_girvan_newman(graph: nx.Graph, seed: Optional[int] = None) -> List[Set[Any]]:
    """
    Compara particiones de Louvain y la mejor de Girvan-Newman basada en modularidad.

    Args:
        graph: El grafo de entrada.
        seed: Semilla aleatoria opcional para Louvain.

    Returns:
        La partición con la modularidad más alta.
    """
    # your code here
    try:
        louvain_fs = nx.community.louvain_communities(graph, seed=seed)
    except TypeError:
        # Para versiones donde el argumento es random_state
        louvain_fs = nx.community.louvain_communities(graph, random_state=seed)
    louvain_partition = [set(c) for c in louvain_fs]
    mod_louvain = nx.community.quality.modularity(graph, louvain_partition)

    # --- 2. Particiones de Girvan-Newman y mejor modularidad ---
    gn_iter = nx.community.girvan_newman(graph)
    best_gn_partition: List[Set[Any]] = []
    best_gn_mod = float("-inf")

    for p_tuple in gn_iter:
        p_list = [set(c) for c in p_tuple]
        if not p_list:
            continue
        mod_gn = nx.community.quality.modularity(graph, p_list)
        if mod_gn > best_gn_mod:
            best_gn_mod = mod_gn
            best_gn_partition = p_list

    # --- 3. Comparar y devolver la mejor partición ---
    # Si empatan en modularidad, devolvemos Louvain
    if best_gn_mod > mod_louvain:
        return best_gn_partition
    else:
        return louvain_partition


# Usar el grafo de Karate Club como ejemplo clásico
K_graph_test1 = nx.karate_club_graph()

student_partition_k1 = compare_louvain_girvan_newman(K_graph_test1, seed=42)
try:
    louvain_ref_k1_fs = nx.community.louvain_communities(K_graph_test1, seed=42)
except TypeError:
    louvain_ref_k1_fs = nx.community.louvain_communities(K_graph_test1, random_state=42)
louvain_ref_k1 = [set(s) for s in louvain_ref_k1_fs]
mod_louvain_ref_k1 = nx.community.quality.modularity(K_graph_test1, louvain_ref_k1)

gn_iter_ref_k1 = nx.community.girvan_newman(K_graph_test1)
best_gn_ref_k1 = []
max_mod_gn_ref_k1 = -1.1
for p_tuple_ref_k1 in gn_iter_ref_k1:
    p_list_ref_k1 = [set(s) for s in p_tuple_ref_k1]
    if not p_list_ref_k1: continue
    mod_ref_k1 = nx.community.quality.modularity(K_graph_test1, p_list_ref_k1)
    if mod_ref_k1 > max_mod_gn_ref_k1:
        max_mod_gn_ref_k1 = mod_ref_k1
        best_gn_ref_k1 = p_list_ref_k1

expected_partition_k1 = louvain_ref_k1
if max_mod_gn_ref_k1 > mod_louvain_ref_k1:
    expected_partition_k1 = best_gn_ref_k1

def normalize_partition(partition: List[Set[Any]]) -> Tuple[frozenset[Any], ...]:
    return tuple(sorted(frozenset(s) for s in partition))

assert normalize_partition(student_partition_k1) == normalize_partition(expected_partition_k1), \
    f"Error: La partición devuelta no es la esperada. Obtenido: {normalize_partition(student_partition_k1)}, Esperado: {normalize_partition(expected_partition_k1)}"
